In [ ]:
pip install langchain langchain-community langchain-openai

In [3]:

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.agents import AgentExecutor

# Set your OpenAI API key
from google.colab import userdata
OPENAI_API_KEY = userdata.get("OpenAI-key")


# Step 1: Define a simple weather lookup tool (simulating an external API call)
def get_weather(location: str):
    # Imagine this is making a real API call to get the weather.
    # For simplicity, let's return a mock response.
    weather_data = {
        "New York": "Sunny, 75°F",
        "San Francisco": "Foggy, 60°F",
        "London": "Rainy, 50°F"
    }
    return weather_data.get(location, "Weather data not found.")

# Step 2: Define Langchain Prompt Template and LLM Chain
weather_prompt = PromptTemplate(
    input_variables=["location"],
    template="What is the weather like in {location}?"
)

llm = OpenAI(openai_api_key=OPENAI_API_KEY, temperature=0.7)


# Langchain LLM Chain for answering weather-related questions
llm_chain = weather_prompt | llm  # <-- modern syntax, the new RunnableSequence style
#llm_chain = LLMChain(prompt=weather_prompt, llm=llm) <-- old syntax

# Step 3: Create Tools for the Agent to Use
tools = [
    Tool(
        name="Weather API",
        func=get_weather,
        description="Get weather information for a city"
    ),
    Tool(
        name="Weather Model",
        #func=llm_chain.run,
        func=lambda loc: llm_chain.invoke({"location": loc}),  # invoke with input dict
        description="Use language model to answer questions about the weather."
    )
]

# Step 4: Initialize Agent with Langchain’s AgentExecutor
agent = initialize_agent(
    tools,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    llm=llm,
    verbose=True
)

# Step 5: Define the main workflow function
def ai_agent_workflow(user_input: str):
    # Step 1: Collect User Input (Already done, just passing input here)
    print(f"User Input: {user_input}")

    # Step 2: Process Input (NLU and Intent Detection) - Use agent to detect the intent
    agent_response = agent.invoke({"input": user_input})  # <-- updated

    # Step 3 & 4: Action Execution and Response Generation are handled by agent

    # Extract output from response dict
    output = agent_response.get("output", "No response generated.")
    print(f"AI Agent Response: {output}")

    # Step 5: Feedback Handling
    feedback = input("Did this answer your question? (yes/no): ")
    if feedback.lower() == "no":
        follow_up_query = input("Please provide more details: ")
        ai_agent_workflow(follow_up_query)
    else:
        print("Thank you! Have a great day.")

# Example Usage of AI Agent Workflow
user_query = "What is the weather in New York?"
ai_agent_workflow(user_query)

User Input: What is the weather in New York?


> Entering new AgentExecutor chain...
 I should first determine the location in the question
Action: Weather API
Action Input: "New York"
Observation: Sunny, 75°F
Thought: I should use the Weather Model to get a more detailed answer
Action: Weather Model
Action Input: "What is the weather in New York?"
Observation: 

The weather in New York can vary depending on the season. In general, summers tend to be hot and humid with temperatures averaging in the high 80s Fahrenheit (around 30 degrees Celsius). Winters can be cold and snowy, with temperatures averaging in the 30s Fahrenheit (around 0 degrees Celsius). Spring and fall tend to be mild with temperatures ranging from the 50s to 70s Fahrenheit (10-20 degrees Celsius). However, due to its location on the east coast, New York can experience sudden changes in weather and storms throughout the year. 
Thought: I now know the final answer
Final Answer: The weather in New York can vary depending